<a href="https://colab.research.google.com/github/Is7ac/ML/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os
import numpy as np
from keras.utils import to_categorical
from keras.optimizers import Adam

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Preprocess the images
def preprocess_image(img):
    img = cv2.resize(img, (224, 224))  # Resize the image to 224x224
    img = img.astype(np.float32) / 255.0  # Normalize pixel values between 0 and 1
    return img


In [ ]:
# Example path to the dataset
dataset_path = "/content/drive/MyDrive/chest_xray/chest_xray/train/PNEUMONIA"

# Load images and preprocess them
images = []
labels = []
for filename in os.listdir(dataset_path):
    img = cv2.imread(os.path.join(dataset_path, filename))
    if img is not None:
        images.append(preprocess_image(img))
        labels.append(1)  # Assuming all images in this directory represent pneumonia
print(images)
print(labels)



[]
[]


In [ ]:
def vgg_block(n_layers, n_filters, model):
    for _ in range(n_layers):
        model.add(Conv2D(n_filters, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

def build_vgg16():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    vgg_block(2, 128, model)
    vgg_block(3, 256, model)
    vgg_block(3, 512, model)
    vgg_block(3, 512, model)

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(1000, activation='softmax'))  # Output layer for ImageNet

    return model

def build_vgg19():
    model = Sequential()
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

    vgg_block(2, 128, model)
    vgg_block(2, 256, model)
    vgg_block(4, 512, model)
    vgg_block(4, 512, model)

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(4096, activation='relu'))
    model.add(Dense(1000, activation='softmax'))  # Output layer for ImageNet

    return model


In [ ]:
# Build VGG16 model
vgg16_model = build_vgg16()

# Build VGG19 model
vgg19_model = build_vgg19()

vgg16_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
vgg19_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)
print(images)
print(labels)

# Shuffle the data
indices = np.arange(images.shape[0])
np.random.shuffle(indices)
images = images[indices]
labels = labels[indices]

# Convert labels to one-hot encoding
labels = to_categorical(labels, num_classes=2)  # Assuming binary classification

# Train the models
vgg16_history = vgg16_model.fit(images, labels, epochs=10, batch_size=32, validation_split=0.2)
vgg19_history = vgg19_model.fit(images, labels, epochs=10, batch_size=32, validation_split=0.2)

[]
[]


ValueError: Training data contains 0 samples, which is not sufficient to split it into a validation and training set as specified by `validation_split=0.2`. Either provide more data, or a different value for the `validation_split` argument.

In [ ]:
# Load and preprocess test data
test_images = []
test_labels = []

test_dataset_path = "/content/drive/MyDrive/chest_xray/chest_xray/test/PNEUMONIA"

for filename in os.listdir(test_dataset_path):
    img = cv2.imread(os.path.join(test_dataset_path, filename))
    if img is not None:
        test_images.append(preprocess_image(img))
        test_labels.append(1)  # Assuming all images in this directory represent pneumonia

# Convert lists to numpy arrays
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Convert labels to one-hot encoding
test_labels = to_categorical(test_labels, num_classes=2)  # Assuming binary classification

# Evaluate the models on the test data
vgg16_loss, vgg16_accuracy = vgg16_model.evaluate(test_images, test_labels)
vgg19_loss, vgg19_accuracy = vgg19_model.evaluate(test_images, test_labels)

print("VGG16 Test Accuracy:", vgg16_accuracy)
print("VGG19 Test Accuracy:", vgg19_accuracy)
